In [5]:
from skimage.transform import pyramid_gaussian
from skimage.util import view_as_windows
from skimage.feature import hog
from skimage.io import imread, imshow
import pickle
import os

In [6]:
def loadModel(path=""):
    """
    Load persisted model
    """
    with open(path+'modelPkl' , 'rb') as files:
        model = pickle.load(files)
    return model

In [7]:
def loadImgs(srcPath):
    """
    Returns:
        - images (List): List of rgb images
    """
    images = []
    for filename in os.listdir(srcPath):
        img = imread(os.path.join(srcPath, filename))
        images.append(img)

    
    return images

In [8]:
def buildPyramid(img, scale=2):
    """
    Returns:
        - imgs(generator): pyramid of images
    """
    # rows, cols, dim = img.shape
    pyramid = tuple(pyramid_gaussian(img, downscale=scale, channel_axis=-1))
    return pyramid

In [9]:
def getWindows(pymImg,shape=(128,64,3)):
    windows = view_as_windows(pymImg, shape, step=(32,16,3))
    return windows

In [10]:
def HOG(img):
    # imshow(img)
    hogImg = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), 
        block_norm='L2', feature_vector=True, channel_axis=-1) # feature vector= true, channel_Axis=2
    return hogImg


In [11]:
def predictPed(img):
    """
    Returns:
        - detection (Bool)
    """
    model = loadModel()
    ans = model.predict(img.reshape(1, -1))
    return ans

In [1]:
def runDetector(img):
    """
    """
    compPretty = []
    complete = []
    # for img in imgs: 
    pyramid = buildPyramid(img)
    print(len(pyramid))
    for level, pymImg in enumerate(pyramid):
        print(pymImg.shape)
        if pymImg.shape >= (128,64,3):
            print("OK")
            window = getWindows(pymImg,(128,64,3))
            rows,cols = window.shape[0], window.shape[1]
            for row in range(rows):
                for col in range(cols):
                    hogWindow = HOG(window[row,col,0])
                    ans = predictPed(hogWindow)
                    if ans == 1:
                        compPretty.append(f"Coordinates {row}, {col}, Level:{level}")
                        complete.append((row,col,level))
        else: 
            break
    return complete


In [ ]:
from skimage.draw import rectangle_perimeter
def drawDetect(img):
    copy = img.copy()
    start = (500,500)
    end = (1000, 200)
    
    rr, cc = rectangle_perimeter(start, end, clip=True, shape=img.shape)
    copy[rr,cc] = [0,255,0]
    
    return copy

In [13]:
img = imread(os.path.join("Central_Park,_NYC_(May_2014)_-_23.JPG"))

In [18]:
x = runDetector(img)
print(x)

13


/Users/katherinegarcia/opt/anaconda3/envs/ml/lib/python3.9/site-packages/skimage/io/_plugins/matplotlib_plugin.py:150: UserWarning: Low image data range; displaying image with stretched contrast.
  lo, hi, cmap = _get_display_range(image)
/Users/katherinegarcia/opt/anaconda3/envs/ml/lib/python3.9/site-packages/skimage/io/_plugins/matplotlib_plugin.py:150: UserWarning: Low image data range; displaying image with stretched contrast.
  lo, hi, cmap = _get_display_range(image)
/Users/katherinegarcia/opt/anaconda3/envs/ml/lib/python3.9/site-packages/skimage/io/_plugins/matplotlib_plugin.py:163: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  ax.get_figure().tight_layout()
/Users/katherinegarcia/opt/anaconda3/envs/ml/lib/python3.9/site-packages/skimage/io/_plugins/matplotlib_plugin.py:163: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  ax.get_figure().tight_l

KeyboardInterrupt: 

In [17]:
x.shape

(128, 64, 3)

In [ ]:
src_path = "/usr/" #read path
dst_path = "/usr/" #write path
